In [ ]:
from pyspark.sql.functions import col, month, year, quarter
from pyspark.sql.types import *
from delta.tables import DeltaTable

class DimDateWrangler:

    @staticmethod
    def extract_silver_df(silver_df):
        return silver_df.dropDuplicates(['ReportedDate']).select(
            col('ReportedDate'), \
            month('ReportedDate').alias('Month'), \
            quarter('ReportedDate').alias('Quarter'), \
            year('ReportedDate').alias('Year'), \
        ) \
        .orderBy('ReportedDate')


    @staticmethod
    def create_delta_table(spark_session, table_name):
        DeltaTable.createIfNotExists(spark_session) \
            .tableName(table_name) \
            .addColumn('ReportedDate', DateType()) \
            .addColumn('Month', ShortType()) \
            .addColumn('Quarter', ShortType()) \
            .addColumn('Year', ShortType()) \
            .execute()


    @staticmethod
    def upsert_delta_table(delta_table, df):

        df_updates = df
        delta_table.alias('existing') \
            .merge(
                df_updates.alias('updates'),
                'existing.ReportedDate = updates.ReportedDate'
            ) \
            .whenMatchedUpdate(set=
                {
                }
            ) \
            .whenNotMatchedInsert(values=
                {
                    'ReportedDate': 'updates.ReportedDate',
                    'Month': 'updates.Month',
                    'Quarter': 'updates.Quarter',
                    'Year': 'updates.Year'
                }
            ) \
            .execute()